In [18]:
#Importando librerías necesarias 

import redshift_connector
import requests
import pandas as pd
from datetime import datetime

In [19]:
#Estableciendo la conexión con la base de datos 

with open("C:/Users/Carolina/Desktop/password.txt",'r') as file:
    pwd= file.read()
host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
db='data-engineer-database'
user='carolina_cargnelutti_coderhouse'

try: 
    conn = redshift_connector.connect(
        host=host,
        database=db,
        password=pwd,
        port=5439,
        user=user      
  )
    cursor = conn.cursor()
    print('Se realizó la conexión correctamente ')

except Exception as e:
    print('No fue posible realizar la conexión')

Se realizó la conexión correctamente 


In [20]:
#Creando la tabla en la base de datos

query="CREATE TABLE IF NOT EXISTS carolina_cargnelutti_coderhouse.weather(id INT primary key NOT NULL, location VARCHAR,region VARCHAR,temperature INTEGER,condition VARCHAR,wind INT,wind_direction VARCHAR,humidity INT,feels_like INTEGER ,visibility INTEGER,updated DATE)"
cursor.execute(query)
conn.commit()

In [21]:
#Comprobando que la tabla esta vacia  

cursor.execute('SELECT * FROM carolina_cargnelutti_coderhouse.weather')
weather_table=cursor.fetchall()
if len(weather_table)>0:
    cursor.execute('TRUNCATE TABLE carolina_cargnelutti_coderhouse.weather')
    print('Se eliminaron correctamente todos los registros de la tabla')
else:
    print('La tabla no tiene registros')

La tabla no tiene registros


In [22]:
#Extrayendo datos de la API

def request(localidad):
    url='http://api.weatherapi.com/v1/forecast.json?key=9bc72106866d42c28da193618240701 &q='+str(localidad)+'&days=1&aqi=yes&alerts=no'
    response=requests.get(url).json()
    return response

def dateFormat(date):
   date_format = '%Y-%m-%d %H:%M' 
   update_date = datetime.strptime(date, date_format)
   return update_date

data={'location':[],'region':[],'temperature':[],'condition':[], 'wind':[], 'wind_direction':[], 'humidity':[],
     'feels_like': [], 'visibility':[],'updated':[]}

locations= ['Buenos Aires','Catamarca','Córdoba','Cataratas del Iguazú','Comodoro Rivadavia','Concordia','El Calafate','Ezeiza','Formosa','Jujuy','Mar del Plata','Mendoza','Neuquen','Paraná','Posadas','Puerto Madryn',
    'Reconquista','Salta','San Carlos de Bariloche', 'Rio Gallegos', 'Termas de Rio Hondo', 'Ushuaia']

for location in locations:
    response=request(location)

    location= response['location']['name']
    region= response['location']['region']
    temperature =response['current']['temp_c']
    feels_like= response['current']['feelslike_c']
    condition = response['current']['condition']['text']
    wind = response['current']['wind_kph']
    wind_direction=response['current']['wind_dir']
    humidity=response['current']['humidity']
    visibility=response['current']['vis_km']
    updated=dateFormat(response['current']['last_updated'])

    data['location'].append(location)
    data['region'].append(region)
    data['temperature'].append(temperature)
    data['condition'].append(condition)
    data['wind'].append(wind)
    data['wind_direction'].append(wind_direction)
    data['humidity'].append(humidity)
    data['feels_like'].append(feels_like)
    data['visibility'].append(visibility)
    data['updated'].append(updated)
    
df=pd.DataFrame(data)

In [23]:
#Limpieza del dataframe:

#Eliminando datos nulos y duplicados:
df.dropna()
df.drop_duplicates(subset='location')

df.head(5)

,location,region,temperature,condition,wind,wind_direction,humidity,feels_like,visibility,updated
0,Buenos Aires,Distrito Federal,27.0,Sunny,20.2,NW,62,27.0,10.0,2024-03-30 12:00:00
1,Catamarca,Catamarca,31.8,Sunny,3.6,ESE,41,32.7,10.0,2024-03-30 12:00:00
2,Córdoba,Cordoba,29.0,Sunny,16.9,N,55,28.4,10.0,2024-03-30 12:00:00
3,Cataratas Del Iguazu,Misiones,31.0,Sunny,4.0,NNW,59,30.9,10.0,2024-03-30 12:00:00
4,Comodoro Rivadavia,Distrito Federal,12.0,Sunny,29.9,W,38,8.8,10.0,2024-03-30 12:00:00


In [24]:
cursor.close()
conn.close()